## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-03-11-48-14 +0000,bbc,Craig the beer-ambassador elephant dies aged 54,https://www.bbc.com/news/articles/c8e5k7p5222o...
1,2026-01-03-11-47-08 +0000,wapo,Live updates: Trump says Maduro captured in ‘l...,https://www.washingtonpost.com/world/2026/01/0...
2,2026-01-03-11-45-28 +0000,nyt,"Live Updates: U.S. Captures Venezuelan Leader,...",https://www.nytimes.com/live/2026/01/03/world/...
3,2026-01-03-11-44-38 +0000,bbc,What we know about US strikes on Venezuela,https://www.bbc.com/news/articles/crmlz7r0zrxo...
4,2026-01-03-11-42-33 +0000,nyt,An Operation in Venezuela,https://www.nytimes.com/2026/01/03/briefing/an...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2373/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
8,trump,25
83,new,23
47,fire,23
103,year,21
44,swiss,21


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
148,2026-01-02-19-35-00 +0000,wsj,"After Swiss New Year’s Fire, Parents Race to F...",https://www.wsj.com/world/europe/after-swiss-n...,108
145,2026-01-02-19-55-19 +0000,nypost,Survivors of Swiss New Year’s Eve fire broke w...,https://nypost.com/2026/01/02/world-news/survi...,108
186,2026-01-02-16-58-00 +0000,wsj,Champagne Sparklers Likely Ignited Ceiling in ...,https://www.wsj.com/world/europe/switzerland-s...,104
17,2026-01-03-10-20-38 +0000,nyt,Swiss Authorities Say Sparklers Probably Cause...,https://www.nytimes.com/live/2026/01/02/world/...,103
144,2026-01-02-19-55-55 +0000,nyt,What to Know About the Swiss Ski Resort Fire o...,https://www.nytimes.com/2026/01/01/world/europ...,100


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
148,108,2026-01-02-19-35-00 +0000,wsj,"After Swiss New Year’s Fire, Parents Race to F...",https://www.wsj.com/world/europe/after-swiss-n...
55,84,2026-01-03-03-00-00 +0000,wsj,The massive welfare-fraud scandal in Minnesota...,https://www.wsj.com/politics/policy/minnesota-...
5,38,2026-01-03-11-37-01 +0000,startribune,"US strikes Venezuela and says its leader, Madu...",https://www.startribune.com/at-least-7-explosi...
194,38,2026-01-02-15-54-50 +0000,nypost,Tesla loses spot as world’s top EV seller to C...,https://nypost.com/2026/01/02/business/tesla-l...
86,34,2026-01-02-23-59-02 +0000,nypost,Iranian-Americans hope mass protests in Iran s...,https://nypost.com/2026/01/02/us-news/iranian-...
168,34,2026-01-02-17-39-02 +0000,nypost,Zelensky makes major changes to his gov’t — ta...,https://nypost.com/2026/01/02/world-news/zelen...
67,33,2026-01-03-01-49-33 +0000,nypost,Coast Guard suspends search for 77-year-old cr...,https://nypost.com/2026/01/02/world-news/coast...
122,30,2026-01-02-21-32-49 +0000,nypost,Newsom will come face-to-face with survivors o...,https://nypost.com/2026/01/02/us-news/newsom-t...
154,30,2026-01-02-19-13-20 +0000,nypost,Elon Musk’s AI chatbot Grok says it generated ...,https://nypost.com/2026/01/02/business/elon-mu...
40,29,2026-01-03-05-09-32 +0000,nypost,DOJ official warns Seattle mayor to leave jour...,https://nypost.com/2026/01/03/us-news/doj-offi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
